# Model Ensembles

In this notebook we will implement stacking ensembles models for online shoppers dataset.


## The Code / Stacking

Our code is saved in the same folder with this report under descion_tree.py and data set is also at the same location called "online_shoppers_intention.csv". We created a set of definitions to help build our model

In [4]:
from Desicion_Tree import *
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV

In [5]:
data_frame_os = read_data_return_frame("online_shoppers_intention.csv")

In [6]:
x, y, class_names, feature_names = preprocess_df(data_frame_os)

The key idea of stacking is to train multiple classifiers and then stack them together using a meta-learner or a voting mechanism.(Module CEGE0004 Week-06 Lecture Slides)
In this case we will use below classifiers:
   * Decision tree classifier that we defined earlier 
   * Naive Bayes Classifiers
       * Bernoulli
       * Gaussian
       * Multinominal
   *  KNeighborsClassifier
    
Let's start with the definition of these classifiers.

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

In [8]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size = 0.25, random_state = 42)

## Desicion Tree

In [9]:
clf = tree.DecisionTreeClassifier(criterion='gini')
params = {'max_depth': [5,6,7,8,None],
          'min_samples_leaf': [5,6,7,8],
          'min_samples_split': [5,6,7,8,9]
         }
grid_search = GridSearchCV(estimator=clf,param_grid=params, scoring='f1')
grid_search.fit(x_train, y_train)
gcv_desc_tr = grid_search.best_estimator_

## Naive Bayes Classifiers

We define a function for different Naive Bayes Classifiers

In [10]:
def create_NB_classifier(x_train, y_train, classifier):
       
    param_grid = [{'alpha': [0.1,0.5, 1.0, 1.5, 5, 10]}]

    if classifier == 'Gaussian':
        return GaussianNB()
    
    elif classifier == "Multinomial":
        classifier = MultinomialNB()
        grid_search = GridSearchCV(classifier, param_grid, cv=5, verbose=2, scoring = 'f1')
        grid_search.fit(x_train, y_train)
        return grid_search.best_estimator_
    
    elif classifier == "Bernoulli":
        classifier = BernoulliNB()        
        grid_search = GridSearchCV(classifier, param_grid, cv=5, verbose=2, scoring = 'f1')
        grid_search.fit(x_train, y_train)
        return grid_search.best_estimator_
    

Now we create each NB classifier

In [11]:
%%capture
gcv_multiN = create_NB_classifier(x_train,y_train,'Multinomial')
gcv_bern = create_NB_classifier(x_train,y_train,'Bernoulli')
clf_gaus = create_NB_classifier(x_train,y_train,'Gaussian');

## KNeighborsClassifier

In [17]:
clf_Knn = KNeighborsClassifier(n_neighbors=1, metric='cosine', weights = 'uniform')

### VotingClassifier

We now define the voting mechanism by using the VotingClassifier of scikit-learn.

In [18]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('dtr', gcv_desc_tr), ('nb_MN', gcv_multiN), ('nb_BE', gcv_bern), ('nb_GA', clf_gaus), ('Knn', clf_Knn)],
    voting='hard')

To use this voting classifier we need to provide an id for each classifier and the classifiers themselves, and a voting parameter which can be set to hard or soft. If this is set to ‘hard’, it uses the predicted class labels for a majority rule voting, if this is set to ‘soft’, it predicts the class label based on the argmax of the sums of the predicted probabilities.

We then fit the voting classifier.(Module CEGE0004 Week-06 Lecture Slides)

In [19]:
voting_clf.fit(x_train, y_train)

VotingClassifier(estimators=[('dtr',
                              DecisionTreeClassifier(min_samples_leaf=5,
                                                     min_samples_split=9)),
                             ('nb_MN', MultinomialNB(alpha=0.1)),
                             ('nb_BE', BernoulliNB()), ('nb_GA', GaussianNB()),
                             ('Knn',
                              KNeighborsClassifier(metric='cosine',
                                                   n_neighbors=1))])

We also need to fit each classifier independently

In [20]:
for clf in (gcv_desc_tr, gcv_multiN, gcv_bern, clf_gaus, clf_Knn):
    clf.fit(x_train, y_train)

### Measures

In [21]:
from sklearn.metrics import accuracy_score

for clf in (gcv_desc_tr, gcv_multiN, gcv_bern, clf_gaus, clf_Knn, voting_clf):
    print(clf.__class__.__name__)
    y_pred = clf.predict(x_train)
    print('\ttrain:', accuracy_score(y_train, y_pred))
    y_pred = clf.predict(x_test)
    print('\ttest:', accuracy_score(y_test, y_pred))

DecisionTreeClassifier
	train: 0.9087271547528928
	test: 0.7872202400259487
MultinomialNB
	train: 0.7261814642586785
	test: 0.7252675964969186
BernoulliNB
	train: 0.8458959662593274
	test: 0.840739539409666
GaussianNB
	train: 0.7621931437222883
	test: 0.7632176451508271
KNeighborsClassifier
	train: 0.9998918568184276
	test: 0.7654881608822576
VotingClassifier
	train: 0.9049421433978587
	test: 0.8297113201427181


Stacking Classifier of Voting Classifier reports 83% on Test data which is higher than every other type of classifiers except Bernoulli which is 84%

### References

Lipani, A. (2021) UCL (University College London)Module CEGE0004 Week-05 Practical Material